In [1]:
%pip install opencv-python tensorflow


Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input

model_path = os.path.join("..", "model", "waste_classifier_v1.keras")
# 1) טען את המודל ששמרת
model = load_model(model_path)

# 2) הגדר רשימת שמות הקטגוריות בהתאמה לסדר שהמודל מחזיר
class_names = ["blue", "brown", "green", "orange", "yellow"]

# 3) הגדר את גודל התמונה שהמודל שלך מצפה לו (למשל 224x224 ל-EfficientNetB3)
IMG_SIZE = (224, 224)

# 4) פתח את המצלמה
cap = cv2.VideoCapture(0)  # 0 = מצלמת ברירת המחדל

# משתנה שיעזור לנו לבדוק כל כמה פריימים נעשה חיזוי
frame_count = 0
prediction_text = ""  # נשמור פה את התחזית הנוכחית

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # כל 10 פריימים נריץ חיזוי
    if frame_count % 10 == 0:
        # עיבוד תמונה מקדימה
        # 1. שנה גודל
        resized = cv2.resize(frame, IMG_SIZE)
        resized = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        resized = preprocess_input(resized.astype('float32'))
        input_tensor = np.expand_dims(resized, axis=0)
        preds = model.predict(input_tensor)
        print("Preds:", preds)
        class_idx = np.argmax(preds[0])
        confidence = preds[0][class_idx]

        # טקסט של התחזית
        prediction_text = f"{class_names[class_idx]} ({confidence:.2f})"

    # הוסף טקסט לפריים (ללא קשר ל־10 פריימים - יישאר עד הבא)
    cv2.putText(
        frame, 
        prediction_text, 
        (20, 40),  # מיקום הטקסט על המסך
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,  # גודל טקסט
        (0, 255, 0),  # צבע הטקסט (B,G,R)
        2,  # עובי הקו
        cv2.LINE_AA
    )

    # הצג את הווידאו
    cv2.imshow("Camera Inference", frame)

    # אם לוחצים על 'q' - צא
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 3s 3s/step
Preds: [[0.16110514 0.09058525 0.49384302 0.03229995 0.22216669]]
1/1 [==============================] - 0s 103ms/step
Preds: [[0.16344993 0.0821607  0.5075503  0.05114403 0.19569506]]
1/1 [==============================] - 0s 109ms/step
Preds: [[0.16797137 0.07517531 0.54122156 0.04090795 0.17472385]]
1/1 [==============================] - 0s 96ms/step
Preds: [[0.1739014  0.07719739 0.49632832 0.04824074 0.2043321 ]]
1/1 [==============================] - 0s 94ms/step
Preds: [[0.13236423 0.08740304 0.46961734 0.03202229 0.27859318]]
1/1 [==============================] - 0s 99ms/step
Preds: [[0.19763666 0.14117596 0.40975997 0.03487911 0.21654834]]
1/1 [==============================] - 0s 116ms/step
Preds: [[0.173859   0.19100103 0.42675495 0.03598325 0.17240177]]
1/1 [==============================] - 0s 104ms/step
Preds: [[0.10703719 0.0428476  0.7758875  0.01769971 0.05652803]]
1/1 [==============================] - 0s 95ms/step
Pr